In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as GraphDataLoader
import matplotlib.pyplot as plt
import skimage

In [ ]:
f1 = 80
f2 = 120
fs = 500
n = np.arange(0, 1000)
n_unif = n[::5]
N = 10_000

tensor_data = np.zeros((N,  len(n_unif),2 , 2))

for s_idx in range(N):
    rand_idx = np.random.permutation(len(n))[:len(n_unif)]
    n_rand = n[rand_idx]
    n_rand = n_unif
    t_rand = n_rand / fs
    phi_rand = np.random.rand() * 2 * np.pi
    # phi_rand = 0
    # t_rand = (np.random.randn(len(n_rand))/3 +  n_rand)/fs
    t_rand = (np.random.rand(len(n_rand)) * n[-1] )/fs
    a_rand = np.cos(2 * np.pi * f1 * t_rand + phi_rand)
    tensor_data[s_idx, :, 0, 0] = t_rand
    tensor_data[s_idx, :, 1, 0] = a_rand
    
    rand_idx = np.random.permutation(len(n))[:len(n_unif)]
    n_rand = n[rand_idx]
    n_rand = n_unif
    t_rand = n_rand / fs
    phi_rand = np.random.rand() * 2 * np.pi
    # phi_rand = 0
    # t_rand = (np.random.randn(len(n_rand))/3 +  n_rand)/fs
    t_rand = (np.random.rand(len(n_rand)) * n[-1] )/fs
    a_rand = np.cos(2 * np.pi * f2 * t_rand + phi_rand)
    tensor_data[s_idx, :, 0, 1] = t_rand
    tensor_data[s_idx, :, 1, 1] = a_rand

# data_matrix = tensor_data.reshape((2,2 * len(n_unif), N))

data_matrix = np.concatenate((tensor_data[...,0], tensor_data[...,1]), axis=0)
labels = np.hstack((np.zeros(N), np.ones(N)))

In [ ]:

plt.stem(tensor_data[0, :, 0, 0], tensor_data[0, :, 1, 0], use_line_collection=True)
plt.show()
plt.stem(tensor_data[0, :, 0, 1], tensor_data[0, :, 1, 1], use_line_collection=True)
plt.show()

In [ ]:
mmm = []
for d in data_matrix:
    ttt = d[:,0]
    aaa = d[:,1]
    idx = np.argsort(ttt)
    ttt = ttt[idx]
    aaa = aaa[idx]

    dif2 = ttt[2:] - ttt[:-2]
    dif2[(np.abs(aaa[:-2] + aaa[1:-1] + aaa[2:]))/3<0.1] = 1e6

    min_idx = np.argmin(dif2)
    
    
    t0, t1, t2 = ttt[min_idx], ttt[min_idx+1], ttt[min_idx+2]
    a0, a1, a2 = aaa[min_idx], aaa[min_idx+1], aaa[min_idx+2]
    d1 = (a1 - a0) / (t1 - t0)
    d2 = (a2 - a1) / (t2 - t1)
    dd = (d2 - d1) / (t2 - t0) * 2
    a_mean = (a0 + a1 + a2) / 3
    mmm.append(-dd/4/np.pi/np.pi/(a_mean))
    # mmm.append(a_mean)
    
thr = skimage.filters.threshold_multiotsu(image=np.array(mmm), classes=2)[0]    
upper_class = np.zeros_like(mmm)
upper_class[np.array(mmm) > thr] = 1
print(np.sum(upper_class == labels) / len(labels))

In [ ]:
import matplotlib.pyplot as plt

# Sample list
data = np.array(mmm)
# Plotting histogram
plt.hist(data, bins=100, edgecolor='black')

# Adding labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of List')

# Displaying the plot
plt.show()

In [ ]:

# Sample list
data = np.array(mmm)
# Plotting histogram
data_upper = data[data > thr]
plt.hist(data_upper, bins=100,color='r')
data_lower = data[data < thr]
plt.hist(data_lower, bins=100,color='b')

# Adding labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of List')

# Displaying the plot
plt.show()

In [ ]:
batch_size = 64
# Convert data and labels to PyTorch tensors
data = torch.tensor(data_matrix, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.long).squeeze()  # Assuming labels are in the shape (N, 1)
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
# Split the data into training and testing sets (80% train, 20% test)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader_graph = GraphDataLoader([Data(x=x[:,1:2], pos=x[:,0:1],y=torch.tensor([y])) for x,y in zip(X_train, y_train)], batch_size=batch_size, shuffle=True)
test_loader_graph = GraphDataLoader([Data(x=x[:,1:2], pos=x[:,0:1],y=torch.tensor([y])) for x,y in zip(X_test, y_test)], batch_size=batch_size)

In [ ]:
class QuantizedTimeSeriesDataset(TensorDataset):
    def __init__(self, tensors, n, fs):
        super(QuantizedTimeSeriesDataset, self).__init__(*tensors)
        self.time_bins = torch.tensor(n/fs)
        
    def __getitem__(self, index):
        data = super(QuantizedTimeSeriesDataset, self).__getitem__(index)
        time = data[0][:, 0]
        amplitude = data[0][:, 1]

        # Quantize the time into bins
        quantized_time = torch.bucketize(time, self.time_bins) - 1

        # Handle multiple amplitudes falling in the same time bin
        binned_amplitude = torch.zeros(len(self.time_bins))
        binned_amplitude.put_(quantized_time, amplitude, accumulate=False)               

        return binned_amplitude, data[1]
    
    def __len__(self):
        return super(QuantizedTimeSeriesDataset, self).__len__()


quantized_train_dataset = QuantizedTimeSeriesDataset([X_train, y_train], n, fs)
train_loader = DataLoader(quantized_train_dataset, batch_size=batch_size, shuffle=True)
quantized_test_dataset = QuantizedTimeSeriesDataset([X_test, y_test], n, fs)
test_loader = DataLoader(quantized_test_dataset, batch_size=batch_size)

In [ ]:

import pytorch_lightning as pl
import torch.nn.functional as F
import torch.nn as nn
import torch
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb

class TimeSeriesCNN(pl.LightningModule):
    def __init__(self, input_length, num_classes):
        super(TimeSeriesCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        # Calculate the size after four max-pooling layers
        def calc_output_size(size, kernel_size=2, stride=2):
            return (size - (kernel_size - 1) - 1) // stride + 1
        
        output_size = input_length
        for _ in range(4):
            output_size = calc_output_size(output_size)
        
        self.fc1 = nn.Linear(128 * output_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = F.relu(self.conv1(x))
        x = F.max_pool1d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool1d(x, 2)
        x = F.relu(self.conv4(x))
        x = F.max_pool1d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, prog_bar=True, logger=True)
        self.log('train_acc', acc, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log('val_loss', loss, prog_bar=True, logger=True)
        self.log('val_acc', acc, prog_bar=True, logger=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

# Example usage
input_length = len(n)  # Length of the quantized time series
num_classes = 2  # Number of classes for classification
model = TimeSeriesCNN(input_length, num_classes)


# Set up W&B logger
wandb_logger = WandbLogger(project='time_series_classification', log_model=False, reinit=True)

# Disable model saving
checkpoint_callback = ModelCheckpoint(save_top_k=0)

# Train the model using PyTorch Lightning
trainer = pl.Trainer(max_epochs=20, logger=wandb_logger, callbacks=[checkpoint_callback])
trainer.fit(model, train_loader, test_loader)
wandb.finish()

In [ ]:
class OrderedTimeSeriesDataset(TensorDataset):
    def __init__(self, tensors):
        super(OrderedTimeSeriesDataset, self).__init__(*tensors)
        
    def __getitem__(self, index):
        data = super(OrderedTimeSeriesDataset, self).__getitem__(index)
        timestamps = data[0][:, 0]

        ordered_time_index = torch.argsort(timestamps)
        data[0][:,0] = data[0][ordered_time_index,0]
        data[0][:,1] = data[0][ordered_time_index,1]

        return data
    
    def __len__(self):
        return super(OrderedTimeSeriesDataset, self).__len__()


ordered_train_dataset = OrderedTimeSeriesDataset([X_train, y_train])
train_loader = DataLoader(ordered_train_dataset, batch_size=batch_size, shuffle=True)
ordered_test_dataset = OrderedTimeSeriesDataset([X_test, y_test])
test_loader = DataLoader(ordered_test_dataset, batch_size=batch_size)

In [ ]:
import pytorch_lightning as pl
import torch.nn.functional as F
import torch.nn as nn
import torch
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

class TimeSeriesLSTM(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, num_classes, num_layers=1):
        super(TimeSeriesLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Get the output of the last time step
        out = self.fc(out)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, prog_bar=True, logger=True)
        self.log('train_acc', acc, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log('val_loss', loss, prog_bar=True, logger=True)
        self.log('val_acc', acc, prog_bar=True, logger=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

# Example usage
input_dim = 2  # Number of features (time and amplitude)
hidden_dim = 64    # Hidden dimension of LSTM
num_classes = 2     # Number of classes for classification
num_layers = 2
model = TimeSeriesLSTM(input_dim, hidden_dim, num_classes, num_layers=num_layers)

# Set up W&B logger
wandb_logger = WandbLogger(project='time_series_lstm_classification', log_model=False, reinit=True)

# Disable model saving
checkpoint_callback = ModelCheckpoint(save_top_k=0)

# Train the model using PyTorch Lightning
trainer = pl.Trainer(max_epochs=10, logger=wandb_logger, callbacks=[checkpoint_callback])
trainer.fit(model, train_loader, test_loader)
wandb.finish()

In [ ]:
X_train[0]ff

In [ ]:

class ImplicitNN(nn.Module):
    def __init__(self, mlp_layers, activation=nn.ReLU()):
        assert mlp_layers[-1] == 1, "Last layer of the mlp must have 1 input channel."
        assert mlp_layers[0] == 1, "First layer of the mlp must have 1 output channel"

        nn.Module.__init__(self)
        self.mlp = nn.ModuleList()
        self.activation = activation

        # create mlp
        in_channels = 1
        for out_channels in mlp_layers[1:]:
            self.mlp.append(nn.Linear(in_channels, out_channels))
            in_channels = out_channels

        # init with trilinear kernel

        # self.init_kernel(num_channels)

    def forward(self, x):
        # create sample of batchsize 1 and input channels 1
        x = x[...,None]

        # apply mlp convolution
        for i in range(len(self.mlp[:-1])):
            x = self.activation(self.mlp[i](x))

        x = self.mlp[-1](x)
        x = x.squeeze()

        return x

    def init_kernel(self, num_channels):
        ts = torch.zeros((1, 2000))
        optim = torch.optim.Adam(self.parameters(), lr=1e-2)

        torch.manual_seed(1)

        for _ in range(1000):  # converges in a reasonable time
            optim.zero_grad()

            ts.uniform_(-1, 1)

            # gt
            gt_values = self.trilinear_kernel(ts, num_channels)

            # pred
            values = self.forward(ts)

            # optimize
            loss = (values - gt_values).pow(2).sum()

            loss.backward()
            optim.step()


    def trilinear_kernel(self, ts, num_channels):
        gt_values = torch.zeros_like(ts)

        gt_values[ts > 0] = (1 - (num_channels-1) * ts)[ts > 0]
        gt_values[ts < 0] = ((num_channels-1) * ts + 1)[ts < 0]

        gt_values[ts < -1.0 / (num_channels-1)] = 0
        gt_values[ts > 1.0 / (num_channels-1)] = 0

        return gt_values

In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import MLP, EdgeConv, DynamicEdgeConv, global_max_pool




class GNN(torch.nn.Module):
    def __init__(self, hidden_size, output_size,k):
        super().__init__()
        self.k = k
        self.aggr = 'max'
        self.conv1 = DynamicEdgeConv(MLP([2 * 2,hidden_size,hidden_size]), self.k, self.aggr)
        self.conv2 = DynamicEdgeConv(MLP([2 * hidden_size,hidden_size,hidden_size]), self.k, self.aggr)
        self.conv3 = DynamicEdgeConv(MLP([2 * hidden_size,hidden_size,hidden_size]), self.k, self.aggr)
        # self.conv2 = EdgeConv(MLP([2 * 64, 128]), aggr)
        self.lin1 = Linear(hidden_size + hidden_size, 1024)

        self.mlp = MLP([ 1024, 512, 256, output_size], dropout=0.5,
                       batch_norm=False)

    def forward(self, data):
        pos = data.pos
        p = data.x
        x0 = torch.cat([pos,p],dim=1)
        if(data.batch is not None):
            batch = data.batch
            x1 = self.conv1(x0, batch)
            x2 = self.conv2(x1, batch)
            # x2 = self.conv2(x2, batch)
            out1 = self.lin1(torch.cat([x1, x2], dim=1))
            out2 = global_max_pool(out1, batch)
        else:       
            x1 = self.conv1(x0)
            x2 = self.conv2(x1)
            # x2 = self.conv2(x2)
            out1 = self.lin1(torch.cat([x1, x2], dim=1))
            out2 = out1.sum(dim=-2, keepdim=out1.dim() == 2)
        out = self.mlp(out2)
        # inters = {'x1': x1, 'x2': x2, 'out1': out1,'out2': out2, 'out': out}
        return F.log_softmax(out, dim=1) #, inters



# Initialize the model, loss function, and optimizer
input_size = np.prod(data.shape[1:])
hidden_size = 64
output_size = len(torch.unique(labels))
model = GNN(hidden_size, output_size,k=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
num_epochs = 150
for epoch in (range(num_epochs)):
    model.train()
    total_correct = 0
    total_samples = 0
    for batch_x in train_loader_graph:
        batch_x = batch_x.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_x.y)
        loss.backward()
        optimizer.step()

        # Calculate training accuracy for this batch
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == batch_x.y).sum().item()
        total_samples += batch_x.y.size(0)

    # Calculate and print training accuracy for the epoch
    train_accuracy = (total_correct / total_samples) * 100
    tqdm.write(f'Epoch [{epoch + 1}/{num_epochs}], Train Accuracy: {train_accuracy:.2f}%')

# Evaluation on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_x in test_loader_graph:
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        _, predicted = torch.max(outputs, 1)
        total += batch_x.y.size(0)
        correct += (predicted == batch_x.y).sum().item()

# Calculate test accuracy
test_accuracy = (correct / total) * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:

# Evaluation on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_x in test_loader_graph:
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        _, predicted = torch.max(outputs, 1)
        total += batch_x.y.size(0)
        correct += (predicted == batch_x.y).sum().item()

# Calculate test accuracy
test_accuracy = (correct / total) * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:

# Create DataLoader for training and testing data
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


class SineActivation(nn.Module):
    def __init__(self):
        super(SineActivation, self).__init__()
        
    def forward(self, x):
        return torch.sin(x)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.sine = SineActivation()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.fc15 = nn.Linear(hidden_size, hidden_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.sine(x)
        x = self.fc15(x)
        x = self.sine(x)
        x = self.fc2(x)
        return torch.softmax(x, dim=1)

# Initialize the model, loss function, and optimizer

input_size = np.prod(data.shape[1:])
hidden_size = 64
output_size = len(torch.unique(labels))
model = MLP(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 200
for epoch in (range(num_epochs)):
    model.train()
    total_correct = 0
    total_samples = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x.view(-1,input_size))
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        # Calculate training accuracy for this batch
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == batch_y).sum().item()
        total_samples += batch_y.size(0)

    # Calculate and print training accuracy for the epoch
    train_accuracy = (total_correct / total_samples) * 100
    tqdm.write(f'Epoch [{epoch + 1}/{num_epochs}], Train Accuracy: {train_accuracy:.2f}%')

# Evaluation on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        outputs = model(batch_x.view(-1,input_size))
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

# Calculate test accuracy
test_accuracy = (correct / total) * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
# Evaluation on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        outputs = model(batch_x.view(-1,input_size))
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

# Calculate test accuracy
test_accuracy = (correct / total) * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')